In [108]:
import pandas as pd
import os, re
from sql import *
from urllib.parse import quote
from sqlalchemy import *
from tools import *
import clickhouse_connect, time

In [109]:
def create_zt_con():
    """
    创建和财务中台的数据库连接
    """
    # 假设你的原始密码包含特殊字符
    password = "1q2w3e!@#"

    # 对密码进行URL编码
    encoded_password = quote(password)

    # 构建DATABASE_URL
    username = "saiyi"
    database = "ssc_1001_db"
    hostname = "rm-uf6um0z4uf7deto67.mysql.rds.aliyuncs.com"
    port = "3306"
    db_type = "mysql+pymysql"

    DATABASE_URL = f"{db_type}://{username}:{encoded_password}@{hostname}:{port}/{database}"

    return create_engine(DATABASE_URL)

In [110]:
def create_zt_mdm_con():
    # 假设你的原始密码包含特殊字符
    password = "1q2w3e!@#"

    # 对密码进行URL编码
    encoded_password = quote(password)

    # 构建DATABASE_U RL
    username = "saiyi"
    database = "ssc_mdm_1001_db"
    hostname = "rm-uf6um0z4uf7deto67.mysql.rds.aliyuncs.com"
    port = "3306"
    db_type = "mysql+pymysql"

    DATABASE_URL = f"{db_type}://{username}:{encoded_password}@{hostname}:{port}/{database}"

    return create_engine(DATABASE_URL)

In [111]:
con = create_zt_con()
con2 = create_zt_mdm_con()

In [112]:
df_detail =  pd.read_sql(detail_sql, con)

---

1. 匹配成本中心

In [115]:
df_other_fields = pd.read_sql(zdbc_sql,con2)
df_other_fields.set_index('VALUE_SET_CODE',inplace=True)
df_other_fields.columns =['成本中心名称']
df_detail = pd.merge(df_detail, df_other_fields, left_on = 'costCenter', right_on='VALUE_SET_CODE', how='left')

2. 预算科目名称

In [117]:
df_other_fields1 = pd.read_sql(zdbc_sql1,con2)
df_other_fields1.set_index('VALUE_SET_CODE',inplace=True)
df_other_fields1.columns =['预算科目名称']
df_detail = pd.merge(df_detail, df_other_fields1, left_on = 'segment', right_on='VALUE_SET_CODE', how='left')

3. 匹配项目阶段名称

In [119]:
df_other_fields2 = pd.read_sql(zdbc_sql2,con2)
df_other_fields2.set_index('VALUE_SET_CODE',inplace=True)
df_other_fields2.columns = ['项目阶段名称']
df_detail = pd.merge(df_detail, df_other_fields2, left_on = 'dimension1', right_on='VALUE_SET_CODE', how='left')

4. 匹配产品名称

In [121]:
df_other_fields2 = pd.read_sql(zdbc_sql3,con2)
df_other_fields2.set_index('VALUE_SET_CODE',inplace=True)
df_other_fields2.columns = ['产品名称']
df_detail = pd.merge(df_detail, df_other_fields2, left_on = 'dimension2', right_on='VALUE_SET_CODE', how='left')

5. 科目类别名称

In [123]:
df_other_fields2 = pd.read_sql(zdbc_sql4,con2)
df_other_fields2.set_index('VALUE_SET_CODE',inplace=True)
df_other_fields2.columns = ['科目类别名称']
df_detail = pd.merge(df_detail, df_other_fields2, left_on = 'segmentCategory', right_on='VALUE_SET_CODE', how='left')

6. 匹配项目名称 

In [125]:
df_project_name = pd.read_sql(sql_text_project_name, con)

In [126]:
df_project_name = pd.read_sql(sql_text_project_name, con)

df_project_name = df_project_name[['projectId','projectName']].copy().drop_duplicates(subset=['projectId'],keep='last')

df_project_name.set_index('projectId',inplace= True)

df_project_name.columns =['项目名称']

df_detail = pd.merge(df_detail, df_project_name, left_on='project',right_on='projectId',how='left')

7. 匹配员工信息

In [128]:
df_emp = pd.read_sql(sql_text_employee, con2)
df_emp = df_emp[['AD_ACCOUNT',  'EMPLOYEE_NUMBER', 'REAL_NAME']].copy()
df_detail = pd.merge(df_detail, df_emp, left_on =['applicant'], right_on = ['AD_ACCOUNT'],how='left' )

8. 匹配部门信息

In [130]:
df_department = pd.read_sql(sql_text_department, con2)
df_department = df_department[['BUSINESS_ID', 'ORGANIZATION_NAME']].copy()
df_detail = pd.merge(df_detail, df_department,left_on='APPLICATION_DEPT',right_on='BUSINESS_ID',how='left')

9. 匹配业务小类

In [132]:
def set_to_str(x):
    try:
        return ",".join(list(x))
    except:
        if pd.isna(x):
            return x
        else:
            return str(x)

In [133]:
def my_clean(input_string):
    try:
        pattern = re.compile(r'"zh-CN":"(.*?)"')
        matches = pattern.findall(input_string)
        return ','.join(matches)
    except:
        if not pd.isna(input_string):
            return str(input_string)
        else:
            return input_string

In [134]:
df_bs_small_kind = pd.read_sql(sql_bs_small_kind, con)
t = pd.read_sql("select CATEGORY_LINE_CODE, CATEGORY_LINE_NAME from mdm_business_category_line", con2)
df_bs_small_kind = pd.merge(df_bs_small_kind, t, left_on='categoryCode', right_on='CATEGORY_LINE_CODE',how='left')
df_bs_small_kind = df_bs_small_kind[['docId', 'segment', 'CATEGORY_LINE_NAME']].copy()
df_bs_small_kind = df_bs_small_kind.groupby(['docId', 'segment']).apply(lambda x: set(x['CATEGORY_LINE_NAME'])).to_frame()
df_bs_small_kind.columns = ['业务小类名称']
df_bs_small_kind['业务小类名称'] = df_bs_small_kind['业务小类名称'].map(set_to_str)
df_bs_small_kind['业务小类名称'] = df_bs_small_kind['业务小类名称'].map(my_clean)
df_detail = pd.merge(df_detail, df_bs_small_kind, on= list(df_bs_small_kind.index.names), how='left')

10. 字段名清洗+ 部门名称清洗+预算主体名称清洗 + 预算操作类型清洗

In [136]:
df_detail.rename(columns = names, inplace=True)

In [137]:
df_detail['部门名称'] = df_detail['部门名称'].map(my_clean)
df_detail['预算主体名称'] = df_detail['预算主体名称'].map(my_clean)

In [138]:
df_detail['预算操作类型'] = df_detail['预算操作类型'].map({"FREEZE":"冻结",
                     "RELEASE":"释放",
                     "DEDUCTION":"扣减",
                     "ADJUST":"调整",
                     "ADD":"新增"                       
                    })

11. 匹配成本中心

In [140]:
def create_otm_con():
    # 假设你的原始密码包含特殊字符
    password = "3s123456"

    # 对密码进行URL编码
    encoded_password = quote(password)

    # 构建DATABASE_URL
    username = "root"
    database = "otm"
    hostname = "10.10.2.51"
    port = "3306"
    db_type = "mysql+pymysql"

    DATABASE_URL = f"{db_type}://{username}:{encoded_password}@{hostname}:{port}/{database}"

    return create_engine(DATABASE_URL)

In [141]:
conx = create_otm_con()

In [142]:
df_cost = pd.read_sql("SELECT * FROM `BU成本中心code映射表` ", con=conx)
df_cost.set_index('ORGANIZATION_CODE', inplace=True)
df_detail = pd.merge(df_detail, df_cost, left_on = ['成本中心'], right_on=['ORGANIZATION_CODE'], how='left')

12. 清洗成本中心、预算科目、申请人、申请人部门

In [144]:
df_detail['成本中心'] = df_detail['成本中心名称'] + '-' +df_detail['成本中心']

df_detail['预算科目'] = df_detail['预算科目名称'] +  df_detail['预算科目']

df_detail['名称'].fillna('无姓名',inplace=True)

df_detail['员工号'].fillna("无员工号", inplace=True)

df_detail['申请人'] = df_detail['名称'] + '-' +df_detail['员工号']

df_detail['申请人部门'] = df_detail['部门名称']

13. 把数据写入clickhouse 

In [146]:
client = clickhouse_connect.get_client(host='10.10.2.51', port=8123, username='root', password='3s123456')

In [147]:
client.command("TRUNCATE TABLE `预算执行报表`")

In [148]:
client.insert_df('预算执行报表',df_detail)

In [181]:
client.command("select count(1) from  `预算执行报表`")

4626808